In [37]:
import hoomd
import flowermd
import numpy as np
import matplotlib as plt
import mbuild as mb
import warnings
warnings.filterwarnings('ignore')
from flowermd.base import Molecule
from flowermd.library import FF_from_file

from flowermd.base import Lattice
from cmeutils.visualize import FresnelGSD
from flowermd.base import Simulation

In [38]:
def espaloma_mol(file_path):
    mol = mb.load(file_path)
    for p in mol.particles():
        p.name = f"_{p.name}"
    return mol

In [39]:
system_file = '../cy5.mol2'
ff_filepath =  "../cy5.xml"

In [40]:
esp_mol = espaloma_mol(system_file)
cy5 = Molecule(num_mols = 2, compound = esp_mol)

In [41]:
cy5.molecules[0].visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [42]:
cy5_ff = FF_from_file(forcefield_files=ff_filepath)
system=Lattice(molecules=cy5, x=1,y=1, n=1)

In [43]:
system.box

Box: Lx=2.149367, Ly=1.980101, Lz=1.946039, xy=0.000000, xz=0.000000, yz=0.000000, 

In [44]:
system.apply_forcefield(
    r_cut=2.5, 
    force_field=cy5_ff, 
    auto_scale=True, 
    scale_charges=True,
    remove_hydrogens=True, #check this
    remove_charges=False) #check this 

In [45]:
snapshot= system.hoomd_snapshot
snapshot.configuration.box *= 3
snapshot.configuration.box

array([18.5255822 , 17.06666374, 16.77308038,  0.        ,  0.        ,
        0.        ])

In [46]:
hoomd_forces = system.hoomd_forcefield
hoomd_forces

In [47]:
working_forces = []
for f in hoomd_forces:
    if not isinstance(f, hoomd.md.special_pair.LJ):
        working_forces.append(f)
working_forces

In [48]:
sim=Simulation(initial_state=snapshot, forcefield=working_forces, reference_values=system.reference_values, gsd_write_freq=100,log_write_freq=100)

Initializing simulation state from a gsd.hoomd.Frame.


In [49]:
lj_force = hoomd_forces[4]

dict(lj_force.params)
#here we see the parameters of the force field t0 and k

{}

In [50]:
sim.box_lengths_reduced

array([18.52558136, 17.06666374, 16.77308083])

In [51]:
system.system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
sim.run_NVT(n_steps=2000000, kT=1.0, tau_kt=0.01)

*Warning*: charge.pppm: RMS error of 0.210864 is probably too high! 0.210864 0.21085


Step 100 of 2000000; TPS: 0.32; ETA: 1736.0 hours, 2.0 minutes
Step 200 of 2000000; TPS: 0.63; ETA: 881.0 hours, 45.0 minutes
Step 300 of 2000000; TPS: 0.95; ETA: 584.0 hours, 42.0 minutes
Step 400 of 2000000; TPS: 1.27; ETA: 437.0 hours, 22.0 minutes
Step 500 of 2000000; TPS: 1.58; ETA: 351.0 hours, 32.0 minutes
Step 600 of 2000000; TPS: 1.9; ETA: 292.0 hours, 19.0 minutes
Step 700 of 2000000; TPS: 2.22; ETA: 250.0 hours, 10.0 minutes
Step 800 of 2000000; TPS: 2.53; ETA: 219.0 hours, 30.0 minutes
Step 900 of 2000000; TPS: 2.85; ETA: 194.0 hours, 51.0 minutes
Step 1000 of 2000000; TPS: 3.17; ETA: 175.0 hours, 10.0 minutes
Step 1100 of 2000000; TPS: 3.48; ETA: 159.0 hours, 33.0 minutes
Step 1200 of 2000000; TPS: 3.8; ETA: 146.0 hours, 7.0 minutes
Step 1300 of 2000000; TPS: 4.11; ETA: 135.0 hours, 5.0 minutes
Step 1400 of 2000000; TPS: 4.43; ETA: 125.0 hours, 19.0 minutes
Step 1500 of 2000000; TPS: 4.74; ETA: 117.0 hours, 7.0 minutes
Step 1600 of 2000000; TPS: 5.06; ETA: 109.0 hours, 42.

In [ ]:
for writer in sim.operations.writers:
    if isinstance(writer, hoomd.write.GSD):
        writer.flush()

In [ ]:
sim_visualizer = FresnelGSD(gsd_file="trajectory.gsd", frame=0, view_axis=(1, 1, 1))
sim_visualizer.view()

In [ ]:
sim_visualizer.frame=-1
sim_visualizer.view()